In [16]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lxml import html
import tqdm
import time

In [6]:
def parse_alexa(website = 'example.ru'):
    
    '''
    
    Функция для загрузки статистики об активности веб-сайтов
    
    param website: имя сайта
    
    '''
    
    url = 'https://alexa.com/siteinfo/' + str(website)
    headers = {
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'
              }
    
    r = requests.get(url, headers = headers)
    
    soup = BeautifulSoup(r.text.encode('utf8'), "lxml")
    if len(soup.findAll('section', {'class': 'note-no-data'})) != 1:
        ranks = soup.findAll('div', {'class': 'rank-row'})
        
        if len(ranks[0].findAll('div')) > 0:
            world_rank = ranks[0].findAll('div')[0].findAll('strong')[0].text
            world_rank = int(''.join(x for x in world_rank if x.isdigit()))
        else:
            world_rank = None

        if len(ranks[1].findAll('div')) > 0:
            country_rank = ranks[1].findAll('div')[0].findAll('strong')[0].text
            country_rank = int(''.join(x for x in country_rank if x.isdigit()))
        else:
            country_rank = None


        link_numbers = soup.findAll('div', {'class': 'box1 box1-med3'})
        link_numbers = link_numbers[0].findAll('span')[0].text
        link_numbers = int(''.join(x for x in link_numbers if x.isdigit()))
    else: 
        country_rank, world_rank, link_numbers = None, None, None
    
    return world_rank, country_rank, link_numbers

----------------------------------

# Load data

In [25]:
# фичи из гугла
data = pd.read_csv('data/Report_Database_plus_features.csv', encoding='cp1251', index_col = 0)
data.head(3)

,name,lat,long,place_id,google_name,google_rating,site,reviews
0,Кемеровская ФФ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Оболенское ФП АО,55.719404,37.611614,ChIJQ4S7OhE1tUYRsayopXzdjU4,"Оболенское, фармацевтическая компания",NaN,http://obolensk.ru/,[]
2,Софарма АО,55.773166,37.527030,ChIJ-zHoPnZJtUYR8D_m_FLE-Ec,РЛС,4.2,http://www.rlsnet.ru/,"[['Проблема с парковкой.', 4, 1485465879], ['С..."


In [17]:
# загрузим статистику веб-сайтов 
for i, row in tqdm.tqdm(data.iterrows(), total = data.shape[0]):
    web = row.site
    
    if pd.isnull(web):
        continue
        
    res = parse_alexa(web.replace('.php', ''))
    
    if res is not None:
        
        world_rank, country_rank, link_numbers = res
        
        data.loc[i, 'world_rank'] = world_rank
        data.loc[i, 'country_rank'] = country_rank
        data.loc[i, 'link_numbers'] = link_numbers
        
    time.sleep(0.5)

100%|████████████████████████████████████████████████████████████████████████████████| 325/325 [12:34<00:00,  2.32s/it]


-------------------------------

# Save

In [18]:
data.to_csv('data/prepared_google.csv', sep=';', encoding='cp1251', index = False)